# Загрузка Pandas и очистка данных

In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('main_task.csv')

# очистка столбца виды кухонь
pattern = re.compile('\w+\s*\w*')
df['Cuisine Style'].fillna('no_information', inplace = True)
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: pattern.findall(x))
# print(df.explode('Cuisine Style')['Cuisine Style'].nunique())
# print(df.explode('Cuisine Style')['Cuisine Style'].value_counts())
df['Cuisine Count'] = df['Cuisine Style'].apply(lambda x: len(x))
# round(df['Cuisine Count'].mean(),2)

# очистка столбца с отзывами
pattern = re.compile('\d+[-]\d+[-]\d+')
df['Reviews'] = df['Reviews'].apply(lambda x: str(x).split(','))
df['Reviews'] = df['Reviews'].apply(lambda x: str(x).replace('/','-'))
df['Reviews'] = df['Reviews'].apply(lambda x: pattern.findall(x))

df['Reviews'] = df['Reviews'].apply(lambda x: pd.to_datetime(x))

In [4]:
pattern

re.compile(r'\d+[-]\d+[-]\d+', re.UNICODE)

In [244]:
d = df.explode('Reviews')['Reviews'].value_counts()
d.sort_index(ascending=False)

2018-02-26    25
2018-02-25    42
2018-02-24    43
2018-02-23    35
2018-02-22    32
              ..
2007-12-08     1
2007-11-23     1
2007-11-21     1
2007-11-14     1
2004-04-21     1
Name: Reviews, Length: 2239, dtype: int64

In [245]:
df['Reviews'][0][1]

Timestamp('2017-11-20 00:00:00')

In [247]:
df['dif_day'] = df['Reviews'].apply(lambda x: x[1]-x[0])

IndexError: index 1 is out of bounds for axis 0 with size 0

In [67]:
df['Price Range'].nunique()

3

In [64]:
df['Price Range'].unique()

array(['$$ - $$$', nan, '$$$$', '$'], dtype=object)

In [65]:
df['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [69]:
df['City'].nunique()

31

In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [53]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [54]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [55]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [48]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [56]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [58]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', round(metrics.mean_absolute_error(y_test, y_pred),2))

MAE: 0.43
